In [1]:
!pip install transformers tokenizers sacrebleu -q
!pip install wandb -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.2 MB/s eta 0:00:00


In [2]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
import torch.nn as nn
import math
import torch.nn.functional as F
from tqdm import tqdm
from tokenizers import Tokenizer
from torch.utils.data import Dataset, DataLoader, Subset
from torch.nn.utils.rnn import pad_sequence
from sacrebleu import corpus_bleu
from google.colab import drive
import os
from torch.cuda.amp import autocast, GradScaler
import wandb

# Định nghĩa các lớp và hàm cần thiết

In [4]:
class BaseSeq2Seq(nn.Module):
    def forward(self, src, tgt, teacher_forcing_ratio=0.5):
        raise NotImplementedError

    def generate(self, src, max_len=100, beam_size=1):
        raise NotImplementedError

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * -(math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        seq_len = x.size(1)
        if seq_len > self.pe.size(1):
            raise ValueError(f"Chiều dài chuỗi đầu vào ({seq_len}) vượt quá max_len của PositionalEncoding ({self.pe.size(1)}). Vui lòng tăng max_len trong PositionalEncoding hoặc giới hạn độ dài câu.")
        x = x + self.pe[:, :seq_len, :]
        return x

In [6]:
class Seq2SeqTransformer(BaseSeq2Seq):
    def __init__(self, input_dim, output_dim, embed_dim, num_heads, hidden_dim, num_layers, src_pad_idx, tgt_pad_idx, device):
        super().__init__()
        self.device = device
        self.src_tok_emb = nn.Embedding(input_dim, embed_dim, padding_idx=src_pad_idx)
        self.tgt_tok_emb = nn.Embedding(output_dim, embed_dim, padding_idx=tgt_pad_idx)
        self.positional_encoding = PositionalEncoding(embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=0.3, batch_first=True)
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=0.3, batch_first=True)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc_out = nn.Linear(embed_dim, output_dim)
        self.src_pad_idx = src_pad_idx
        self.tgt_pad_idx = tgt_pad_idx

    def make_tgt_mask(self, tgt):
        tgt_len = tgt.size(1)
        mask = nn.Transformer.generate_square_subsequent_mask(tgt_len).to(self.device)
        return mask

    def forward(self, src, tgt, teacher_forcing_ratio=1.0):
        src_padding_mask = (src == self.src_pad_idx)
        tgt_padding_mask = (tgt == self.tgt_pad_idx)

        tgt_mask = self.make_tgt_mask(tgt).bool()

        src_embed = self.positional_encoding(self.src_tok_emb(src))
        tgt_embed = self.positional_encoding(self.tgt_tok_emb(tgt))

        memory = self.encoder(src_embed, src_key_padding_mask=src_padding_mask)

        output = self.decoder(
            tgt_embed,
            memory,
            tgt_mask=tgt_mask,
            tgt_key_padding_mask=tgt_padding_mask,
            memory_key_padding_mask=src_padding_mask
        )
        output = self.fc_out(output)
        return output
    def generate(self, src, max_len=100, beam_size=1):
        self.eval()
        with torch.no_grad():
            with autocast():
                src_padding_mask = (src == self.src_pad_idx).to(self.device)

                src_embed = self.positional_encoding(self.src_tok_emb(src))
                memory = self.encoder(src_embed, src_key_padding_mask=src_padding_mask)

                batch_size = src.size(0)
                sos_id = target_tokenizer.token_to_id("<sos>")
                eos_id = target_tokenizer.token_to_id("<eos>")
                pad_id = self.tgt_pad_idx

                output_sequences = torch.full((batch_size, 1), fill_value=sos_id, dtype=torch.long, device=self.device)

                finished_sequences = torch.zeros(batch_size, dtype=torch.bool, device=self.device)

                for _ in range(max_len):
                    if finished_sequences.all():
                        break

                    tgt_input = output_sequences

                    tgt_embed = self.positional_encoding(self.tgt_tok_emb(tgt_input))
                    tgt_mask = self.make_tgt_mask(tgt_input).bool()

                    decoder_output = self.decoder(
                        tgt_embed,
                        memory,
                        tgt_mask=tgt_mask,
                        memory_key_padding_mask=src_padding_mask
                    )

                    output_logits = self.fc_out(decoder_output[:, -1, :])
                    log_probs = F.log_softmax(output_logits, dim=-1)

                    next_token_log_probs, next_token_ids = log_probs.max(dim=-1)

                    next_token_ids = torch.where(finished_sequences, output_sequences[:, -1], next_token_ids)

                    finished_sequences = finished_sequences | (next_token_ids == eos_id)

                    output_sequences = torch.cat([output_sequences, next_token_ids.unsqueeze(-1)], dim=-1)

                final_results = []
                for seq in output_sequences.tolist():
                    clean_seq = []
                    start_idx = 1 if seq[0] == sos_id else 0
                    for token_id in seq[start_idx:]:
                        if token_id == eos_id:
                            break
                        clean_seq.append(token_id)
                    final_results.append(clean_seq)

                if not final_results:
                    return torch.full((batch_size, 1), fill_value=eos_id, dtype=torch.long, device=self.device)

                max_len_out = max(len(seq) for seq in final_results)
                output_tensor = torch.full(
                    (batch_size, max_len_out),
                    fill_value=pad_id,
                    dtype=torch.long,
                    device=self.device
                )
                for i, seq in enumerate(final_results):
                    output_tensor[i, :len(seq)] = torch.tensor(seq, dtype=torch.long, device=self.device)

                return output_tensor

In [ ]:
class NoamScheduler:
    def __init__(self, optimizer, model_size, warmup_steps):
        self.optimizer = optimizer
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.current_step = 0
        self._rate = 0

    def step(self):
        self.current_step += 1
        lr = self.model_size**(-0.5) * min(self.current_step**(-0.5), self.current_step * self.warmup_steps**(-1.5))
        for p in self.optimizer.param_groups:
            p['lr'] = lr
        self._rate = lr

    def zero_grad(self):
        self.optimizer.zero_grad()

    def get_last_lr(self):
        return self.optimizer.param_groups[0]['lr']

In [ ]:

global_wandb_step = 0

def train_epoch(model, dataloader, optimizer, criterion, device, scaler, clip_value=1.0, accumulation_steps=1):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    loop = tqdm(dataloader, total=len(dataloader), desc="Huấn luyện", leave=True, colour="green")

    global global_wandb_step

    for i, batch in enumerate(loop):
        src = batch['src'].to(device)
        tgt = batch['tgt'].to(device)
        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]

        # Mixed precision training
        with autocast():
            output = model(src, tgt_input)
            output = output.reshape(-1, output.shape[-1])
            tgt_output = tgt_output.reshape(-1)
            loss = criterion(output, tgt_output)
            loss = loss / accumulation_steps

        scaler.scale(loss).backward()

        if (i + 1) % accumulation_steps == 0:
            scaler.unscale_(optimizer.optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip_value)
            scaler.step(optimizer.optimizer)
            scaler.update()
            optimizer.zero_grad()

            global_wandb_step += 1
            wandb.log({"Train Batch Loss": loss.item() * accumulation_steps,
                       "Learning Rate": optimizer.optimizer.param_groups[0]['lr']},
                       step=global_wandb_step)
        total_loss += loss.item() * accumulation_steps
        loop.set_postfix(loss=loss.item() * accumulation_steps)

    return total_loss / len(dataloader)


In [9]:

def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        with autocast():
            loop = tqdm(dataloader, total=len(dataloader), desc="Đánh giá", leave=True, colour="blue")
            for batch in loop:
                src = batch['src'].to(device)
                tgt = batch['tgt'].to(device)
                if tgt.dim() == 1:
                    tgt = tgt.unsqueeze(0)
                tgt_input = tgt[:, :-1]
                tgt_output = tgt[:, 1:]

                output = model(src, tgt_input)
                output = output.reshape(-1, output.shape[-1])
                tgt_output = tgt_output.reshape(-1)
                loss = criterion(output, tgt_output)
                total_loss += loss.item()
                loop.set_postfix(loss=loss.item())
    return total_loss / len(dataloader)


In [10]:

def get_model(name, config):
    if name == 'transformer':
        model_config = {
            "input_dim": config["input_dim"],
            "output_dim": config["output_dim"],
            "embed_dim": config["embed_dim"],
            "num_heads": config["num_heads"],
            "hidden_dim": config["hidden_dim"],
            "num_layers": config["num_layers"],
            "src_pad_idx": config["src_pad_idx"],
            "tgt_pad_idx": config["tgt_pad_idx"],
            "device": config["device"]
        }
        return Seq2SeqTransformer(**model_config)
    else:
        raise ValueError(f"Tên mô hình không xác định: {name}")


In [ ]:

def compute_bleu(model, valid_dataset, device, source_tokenizer, target_tokenizer, config, num_examples_to_print=5, num_subsets=10):
    model.eval()
    overall_hypotheses = []
    overall_references = []
    overall_original_sources = []
    bleu_scores_per_subset = []

    tgt_pad_idx = target_tokenizer.token_to_id("<pad>")
    tgt_sos_idx = target_tokenizer.token_to_id("<sos>")
    tgt_eos_idx = target_tokenizer.token_to_id("<eos>")

    src_pad_idx = source_tokenizer.token_to_id("<pad>")
    src_sos_idx = source_tokenizer.token_to_id("<sos>")
    src_eos_idx = source_tokenizer.token_to_id("<eos>")

    total_examples = len(valid_dataset)
    subset_size = total_examples // num_subsets

    with torch.no_grad():
        with autocast():
            for i in range(num_subsets):
                start_idx = i * subset_size
                end_idx = (i + 1) * subset_size
                if i == num_subsets - 1:
                    end_idx = total_examples

                current_subset_indices = list(range(start_idx, end_idx))
                current_subset = Subset(valid_dataset, current_subset_indices)
                current_dataloader = DataLoader(current_subset, batch_size=64, shuffle=False, collate_fn=collate_fn, num_workers=2)

                subset_hypotheses = []
                subset_references = []
                subset_original_sources = []

                loop = tqdm(current_dataloader, desc=f"Tính BLEU Tập con {i+1}/{num_subsets}", leave=False, colour="magenta")
                for batch in loop:
                    src = batch['src'].to(device)
                    tgt = batch['tgt'].to(device)

                    output_ids = model.generate(src, max_len=config['max_seq_len_generation'], beam_size=1)

                    for src_ids_sample, ref_ids_sample, hyp_ids_sample in zip(src.tolist(), tgt.tolist(), output_ids.tolist()):
                        src_text = source_tokenizer.decode(src_ids_sample, skip_special_tokens=True)
                        subset_original_sources.append(src_text)

                        ref_text = target_tokenizer.decode(ref_ids_sample, skip_special_tokens=True)
                        subset_references.append(ref_text)

                        hyp_text = target_tokenizer.decode(hyp_ids_sample, skip_special_tokens=True)
                        subset_hypotheses.append(hyp_text)

                if subset_hypotheses and subset_references:
                    subset_bleu = corpus_bleu(subset_hypotheses, [subset_references])
                    bleu_scores_per_subset.append(subset_bleu.score)
                    print(f"  - Điểm BLEU tập thứ {i+1}: {subset_bleu.score:.2f}")
                    wandb.log({f"BLEU_Subset_{i+1}": subset_bleu.score}, step=global_wandb_step)

                    overall_hypotheses.extend(subset_hypotheses)
                    overall_references.extend(subset_references)
                    overall_original_sources.extend(subset_original_sources)
                else:
                    print(f"  - Tập con thứ {i+1} không có đủ dữ liệu để tính BLEU.")

    if overall_hypotheses and overall_references:
        overall_bleu = corpus_bleu(overall_hypotheses, [overall_references])
        print(f"\n- Điểm BLEU Trung bình (tất cả tập con): {sum(bleu_scores_per_subset) / len(bleu_scores_per_subset):.2f}")
        print(f"- Điểm BLEU Tổng thể: {overall_bleu.score:.2f}")
    else:
        print("\nKhông có dữ liệu tổng thể để tính BLEU.")
        return 0.0

    print("\n- Bản dịch mẫu:")
    for i in range(min(num_examples_to_print, len(overall_hypotheses))):
        print(f"Gốc (SRC): {overall_original_sources[i]}")
        print(f"Tham chiếu (REF): {overall_references[i]}")
        print(f"Dịch máy (HYP): {overall_hypotheses[i]}")
        print("-" * 50)

    return overall_bleu.score


In [12]:
class TranslationDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src = src_data
        self.tgt = tgt_data

    def __len__(self):
        return len(self.src)

    def __getitem__(self, idx):
        return {
            'src': torch.tensor(self.src[idx], dtype=torch.long),
            'tgt': torch.tensor(self.tgt[idx], dtype=torch.long)
        }


In [ ]:
def collate_fn(batch):
    src_pad_id = source_tokenizer.token_to_id("<pad>")
    tgt_pad_id = target_tokenizer.token_to_id("<pad>")

    src_batch = [item['src'] for item in batch]
    tgt_batch = [item['tgt'] for item in batch]

    src_pad = pad_sequence(src_batch, batch_first=True, padding_value=src_pad_id)
    tgt_pad = pad_sequence(tgt_batch, batch_first=True, padding_value=tgt_pad_id)
    return {
        'src': src_pad,
        'tgt': tgt_pad
    }

# CẤU HÌNH VÀ TẢI DỮ LIỆU/TOKENIZER TRÊN COLAB

In [ ]:
drive.mount('/content/drive')

GOOGLE_DRIVE_PATH = '/content/drive/MyDrive/NMT_Project_Files'

try:
    source_tokenizer = Tokenizer.from_file(os.path.join(GOOGLE_DRIVE_PATH, '300kenglish_bpe_bytelevel.json'))
    target_tokenizer = Tokenizer.from_file(os.path.join(GOOGLE_DRIVE_PATH, '300kvietnamese_bpe_bytelevel.json'))

    from tokenizers.processors import BertProcessing
    source_tokenizer.post_processor = BertProcessing(
        sep=("<eos>", source_tokenizer.token_to_id("<eos>")),
        cls=("<sos>", source_tokenizer.token_to_id("<sos>")),
    )
    target_tokenizer.post_processor = BertProcessing(
        sep=("<eos>", target_tokenizer.token_to_id("<eos>")),
        cls=("<sos>", target_tokenizer.token_to_id("<sos>")),
    )

    if source_tokenizer.token_to_id("<unk>") is None:
        source_tokenizer.add_special_tokens(["<unk>"])
    if target_tokenizer.token_to_id("<unk>") is None:
        target_tokenizer.add_special_tokens(["<unk>"])


except Exception as e:
    print(f"Lỗi khi tải Tokenizer: {e}")
    print("Vui lòng kiểm tra đường dẫn và tên file tokenizer cũng như các token đặc biệt.")
    exit()
src_ids = []
tgt_ids = []
src_ids_validate = []
tgt_ids_validate = []

temp_src_ids = []
temp_tgt_ids = []
try:
    with open(os.path.join(GOOGLE_DRIVE_PATH, "100kenglish.txt"), "r", encoding="utf-8") as f_src, \
         open(os.path.join(GOOGLE_DRIVE_PATH, "100kvietnam.txt"), "r", encoding="utf-8") as f_tgt:
        for i, (src_line, tgt_line) in enumerate(zip(f_src, f_tgt)):
            src_line = src_line.strip()
            tgt_line = tgt_line.strip()
            if src_line and tgt_line:
                src_output = source_tokenizer.encode(src_line)
                src_id = src_output.ids
                temp_src_ids.append(src_id)

                tgt_output = target_tokenizer.encode(tgt_line)
                tgt_id = tgt_output.ids
                temp_tgt_ids.append(tgt_id)
            elif not src_line and not tgt_line:
                continue
            else:
                print(f"Cảnh báo: Dòng {i+1} trong tập huấn luyện có sự không khớp (một dòng rỗng, một không rỗng). Bỏ qua cặp này.")
    src_ids = temp_src_ids
    tgt_ids = temp_tgt_ids
    if len(src_ids) != len(tgt_ids):
        print(f"CẢNH BÁO: Số lượng câu nguồn ({len(src_ids)}) và câu đích ({len(tgt_ids)}) không khớp sau khi tiền xử lý.")
except FileNotFoundError as e:
    print(f"Lỗi FileNotFoundError khi tải dữ liệu huấn luyện: {e}. Vui lòng kiểm tra đường dẫn.")
    exit()
except Exception as e:
    print(f"Lỗi không xác định khi tải dữ liệu huấn luyện: {e}")
    exit()

temp_src_ids_validate = []
temp_tgt_ids_validate = []
try:
    with open(os.path.join(GOOGLE_DRIVE_PATH, "english_validate.txt"), "r", encoding="utf-8") as f_src_val, \
         open(os.path.join(GOOGLE_DRIVE_PATH, "vietnam_validate.txt"), "r", encoding="utf-8") as f_tgt_val:
        for i, (src_line_val, tgt_line_val) in enumerate(zip(f_src_val, f_tgt_val)):
            src_line_val = src_line_val.strip()
            tgt_line_val = tgt_line_val.strip()
            if src_line_val and tgt_line_val:
                src_output_val = source_tokenizer.encode(src_line_val)
                src_id_val = src_output_val.ids
                temp_src_ids_validate.append(src_id_val)

                tgt_output_val = target_tokenizer.encode(tgt_line_val)
                tgt_id_val = tgt_output_val.ids
                temp_tgt_ids_validate.append(tgt_id_val)
            elif not src_line_val and not tgt_line_val:
                continue
            else:
                print(f"Cảnh báo: Dòng validation {i+1} có sự không khớp (một dòng rỗng, một không rỗng). Bỏ qua cặp này.")
    src_ids_validate = temp_src_ids_validate
    tgt_ids_validate = temp_tgt_ids_validate
    if len(src_ids_validate) != len(tgt_ids_validate):
        print(f"CẢNH BÁO: Số lượng câu nguồn validation ({len(src_ids_validate)}) và câu đích validation ({len(tgt_ids_validate)}) không khớp sau khi tiền xử lý.")
except FileNotFoundError as e:
    print(f"Lỗi FileNotFoundError khi tải dữ liệu validation: {e}. Vui lòng kiểm tra đường dẫn.")
    exit()
except Exception as e:
    print(f"Lỗi không xác định khi tải dữ liệu validation: {e}")
    exit()

print("\n--- Thông tin về dữ liệu đã tải ---")
print(f"Số lượng cặp câu huấn luyện: {len(src_ids)}")
print(f"Số lượng cặp câu validation: {len(src_ids_validate)}")

avg_src_len = sum(len(s) for s in src_ids) / len(src_ids) if len(src_ids) > 0 else 0
max_src_len = max(len(s) for s in src_ids) if len(src_ids) > 0 else 0
avg_tgt_len = sum(len(t) for t in tgt_ids) / len(tgt_ids) if len(tgt_ids) > 0 else 0
max_tgt_len = max(len(t) for t in tgt_ids) if len(tgt_ids) > 0 else 0

print(f"Độ dài câu nguồn trung bình: {avg_src_len:.2f}")
print(f"Độ dài câu nguồn tối đa: {max_src_len}")
print(f"Độ dài câu đích trung bình: {avg_tgt_len:.2f}")
print(f"Độ dài câu đích tối đa: {max_tgt_len}")


Mounted at /content/drive

--- Thông tin về dữ liệu đã tải ---
Số lượng cặp câu huấn luyện: 100001
Số lượng cặp câu validation: 10001
Độ dài câu nguồn trung bình: 22.68
Độ dài câu nguồn tối đa: 202
Độ dài câu đích trung bình: 25.42
Độ dài câu đích tối đa: 259


In [ ]:

dataset = TranslationDataset(src_ids, tgt_ids)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=collate_fn, num_workers=2)

validate = TranslationDataset(src_ids_validate, tgt_ids_validate)
full_valid_dataloader = DataLoader(validate, batch_size=64, shuffle=False, collate_fn=collate_fn, num_workers=2)

# Cấu hình mô hình

In [ ]:
config = {
    "input_dim": source_tokenizer.get_vocab_size(),
    "output_dim": target_tokenizer.get_vocab_size(),
    "embed_dim": 256,
    "hidden_dim": 512,
    "num_layers": 6,
    "num_heads": 8,
    "src_pad_idx": source_tokenizer.token_to_id("<pad>"),
    "tgt_pad_idx": target_tokenizer.token_to_id("<pad>"),
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "warmup_steps": 4000,
    "beam_size": 1,
    "accumulation_steps": 4,
    "max_seq_len_generation": 100
}
num_epochs = 30

wandb.login() 
wandb.init(project="NMT_Transformer_PhoMT", config=config)

wandb.config.update({
    "source_vocab_size": config["input_dim"],
    "target_vocab_size": config["output_dim"],
    "model_embedding_dim": config["embed_dim"],
    "model_num_heads": config["num_heads"],
    "model_feedforward_dim": config["hidden_dim"],
    "model_num_layers": config["num_layers"],
    "training_warmup_steps": config["warmup_steps"],
    "inference_beam_size": config["beam_size"],
    "training_gradient_accumulation_steps": config["accumulation_steps"],
    "training_device": config["device"].type,
    "max_sequence_length_generation": config["max_seq_len_generation"]
})


model = get_model("transformer", config).to(config["device"])

total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n--- Thông số Mô hình Transformer ---")
print(f"Embedding Dimension (embed_dim): {config['embed_dim']}")
print(f"Number of Heads (num_heads): {config['num_heads']}")
print(f"Feedforward Dimension (hidden_dim): {config['hidden_dim']}")
print(f"Number of Layers (num_layers): {config['num_layers']}")
print(f"Tổng số tham số có thể huấn luyện: {total_params:,}")


try:
    model_path = os.path.join(GOOGLE_DRIVE_PATH, "best_model.pt")
    model.load_state_dict(torch.load(model_path, map_location=config["device"]))
    print("\nĐã tải 'best_model.pt' để tiếp tục huấn luyện.")
except FileNotFoundError:
    print("\nKHÔNG TÌM THẤY FILE 'best_model.pt'. Mô hình sẽ bắt đầu huấn luyện từ đầu với các trọng số ngẫu nhiên.")
except Exception as e:
    print(f"\nLỗi khi tải mô hình: {e}")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)
criterion = torch.nn.CrossEntropyLoss(ignore_index=config["tgt_pad_idx"])
scaler = GradScaler()
scheduler = NoamScheduler(optimizer, model_size=config["embed_dim"], warmup_steps=config["warmup_steps"])

patience = 6
counter = 0
best_val_loss = float('inf')


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tonthanhdat05092000 (tonthanhdat05092000-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



--- Thông số Mô hình Transformer ---
Embedding Dimension (embed_dim): 256
Number of Heads (num_heads): 8
Feedforward Dimension (hidden_dim): 512
Number of Layers (num_layers): 6
Tổng số tham số có thể huấn luyện: 20,211,328

Đã tải 'best_model.pt' để tiếp tục huấn luyện.


/tmp/ipython-input-16-727646305.py:62: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


# Huấn luyện

In [ ]:
for epoch in range(num_epochs):
    print(f'\nEpoch: {epoch+1}/{num_epochs}')

    train_loss = train_epoch(model, dataloader, scheduler, criterion, config["device"], scaler,
                             clip_value=1.0, accumulation_steps=config["accumulation_steps"])

    val_loss = validate_epoch(model, full_valid_dataloader, criterion, config["device"])

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")

    wandb.log({
        "Epoch": epoch + 1,
        "Train Loss (Epoch)": train_loss,
        "Validation Loss (Epoch)": val_loss,
        "Learning Rate (Epoch End)": scheduler.get_last_lr()
    }, step=global_wandb_step)

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        save_path = os.path.join(GOOGLE_DRIVE_PATH, 'new_best_model.pt')
        torch.save(model.state_dict(), save_path)
        print(f"Đã lưu mô hình tốt nhất mới tại epoch {epoch+1} với val_loss: {best_val_loss:.4f} vào '{save_path}'")
    else:
        counter += 1
        print(f"Val Loss không cải thiện trong {counter}/{patience} epoch.")
        if counter >= patience:
            print('Early stopping do val_loss không cải thiện.')
            break



Epoch: 1/30


Huấn luyện:   0%|          | 0/1563 [00:00<?, ?it/s]/tmp/ipython-input-8-1141824594.py:18: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Huấn luyện: 100%|██████████| 1563/1563 [02:26<00:00, 10.68it/s, loss=1.99]
/tmp/ipython-input-9-218587686.py:5: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Đánh giá: 100%|██████████| 157/157 [00:04<00:00, 34.82it/s, loss=2.82]


Train Loss: 1.7556 | Val Loss: 3.0087
Đã lưu mô hình tốt nhất mới tại epoch 1 với val_loss: 3.0087 vào '/content/drive/MyDrive/NMT_Project_Files/new_best_model.pt'

Epoch: 2/30


Đánh giá: 100%|██████████| 157/157 [00:03<00:00, 41.97it/s, loss=2.82]


Train Loss: 1.7423 | Val Loss: 3.0098
Val Loss không cải thiện trong 1/6 epoch.

Epoch: 3/30


...

Train Loss: 1.6934 | Val Loss: 3.0149
Val Loss không cải thiện trong 4/6 epoch.

Epoch: 9/30


Đánh giá: 100%|██████████| 157/157 [00:03<00:00, 39.68it/s, loss=2.79]


Train Loss: 1.6881 | Val Loss: 3.0118
Val Loss không cải thiện trong 5/6 epoch.

Epoch: 10/30


Đánh giá: 100%|██████████| 157/157 [00:03<00:00, 39.96it/s, loss=2.8]

Train Loss: 1.6810 | Val Loss: 3.0100
Val Loss không cải thiện trong 6/6 epoch.
Early stopping do val_loss không cải thiện.


# Đánh giá BLEU trên mô hình tốt nhất sau khi huấn luyện

In [ ]:

try:
    final_best_model_path = os.path.join(GOOGLE_DRIVE_PATH, 'new_best_model.pt')
    if os.path.exists(final_best_model_path):
        model.load_state_dict(torch.load(final_best_model_path, map_location=config["device"]))
        print(f"Đã tải mô hình tốt nhất từ '{final_best_model_path}' để tính BLEU.")

        final_bleu_score = compute_bleu(model, validate, config["device"],
                                        source_tokenizer, target_tokenizer, config, num_examples_to_print=10)

    else:
        print(f"Không tìm thấy file mô hình tốt nhất tại '{final_best_model_path}'. Không thể tính BLEU.")
except Exception as e:
    print(f"Lỗi khi tải lại mô hình hoặc tính BLEU: {e}")


/tmp/ipython-input-11-1344781054.py:63: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Đã tải mô hình tốt nhất từ '/content/drive/MyDrive/NMT_Project_Files/new_best_model.pt' để tính BLEU.


Tính BLEU Tập con 1/10:   0%|          | 0/16 [00:00<?, ?it/s]/tmp/ipython-input-6-3207906614.py:44: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  - Điểm BLEU tập thứ 1: 20.80


  - Điểm BLEU tập thứ 2: 24.01


...

  - Điểm BLEU tập thứ 6: 17.82


  - Điểm BLEU tập thứ 7: 13.76


  - Điểm BLEU tập thứ 8: 15.91


  - Điểm BLEU tập thứ 9: 14.22


  - Điểm BLEU tập thứ 10: 14.67

- Điểm BLEU Trung bình (tất cả tập con): 17.39
- Điểm BLEU Tổng thể: 19.03

- Bản dịch mẫu:
Gốc (SRC):  brother albert barnett and his wife, sister susan barnett, from the west congregation in tuscaloosa, alabama
Tham chiếu (REF):  anh albert barnett và chị susan barnett, thuộc hội thánh west ở tuscaloosa, alabama
Dịch máy (HYP):  anh trai rượu và vợ của anh, chị gái rút lui, từ tây âu, alabama, alabama.
--------------------------------------------------
Gốc (SRC):  severe storms ripped through parts of the southern and midwestern united states on january 11 and 12, 2020.
Tham chiếu (REF):  ngày 11 và 12-1-2020, những cơn bão lớn đã quét qua và phá huỷ nhiều vùng ở miền nam và miền trung hoa kỳ.
Dịch máy (HYP):  cảnh quay vô tình xuyên qua những phần của nam và mỹ ở 11  11  năm 2020 và năm 2020.
--------------------------------------------------
Gốc (SRC):  two days of heavy rain, high winds, and numerous tornadoes caused major damage across multiple st

In [ ]:

wandb.finish();

BLEU_Subset_1,▁
BLEU_Subset_10,▁
BLEU_Subset_2,▁
BLEU_Subset_3,▁
BLEU_Subset_4,▁
BLEU_Subset_5,▁
BLEU_Subset_6,▁
BLEU_Subset_7,▁
BLEU_Subset_8,▁
BLEU_Subset_9,▁
Epoch,▁▂▃▃▄▅▆▆▇█
